In [1]:
### plotting chic_abc_epi sushi version
### Fig.5 D,E and Fig. 6A

library(data.table)
library(Chicago)
library(ggplot2)
library(GenomicRanges)

library(Sushi)
library(biomaRt)


Warning message:
“package ‘Chicago’ was built under R version 3.6.3”
Warning message:
“replacing previous import ‘vctrs::data_frame’ by ‘tibble::data_frame’ when loading ‘dplyr’”


Welcome to CHiCAGO - version 1.14.0

If you are new to CHiCAGO, please consider reading the vignette through the command: vignette("Chicago").

NOTE: Default values of tlb.minProxOEPerBin and tlb.minProxB2BPerBin changed as of Version 1.1.5. No action is required unless you specified non-default values, or wish to re-run the pipeline on old chicagoData objects. See news(package="Chicago")

Warning message:
“package ‘GenomicRanges’ was built under R version 3.6.3”
Loading required package: stats4

Loading required package: BiocGenerics

Warning message:
“package ‘BiocGenerics’ was built under R version 3.6.3”
Loading required package: parallel


Attaching package: ‘BiocGenerics’


The following objects are masked from ‘package:parallel’:

    clusterApply, clusterApplyLB, clusterCall, clusterEvalQ,
    cluste

In [2]:
### loading chicago/abc peakmatrix

ilc_chic_abc_peakm <- fread("ILC3_chicago_fres_5kb_abc_fres_extended_peakm.txt")


In [3]:
### loading epi files

k27ac <- "ILC3_H3K27ac_peaks.bed"
atac <- "ILC3_ATAC_peaks.bed"

k36me3 <- "GSE50893_SNYDER_HG38_GM12878_H3K36ME3_VS_SNYDER_HG19_GM12878_INPUT_reconcile.dedup_peaks.encodePeak.txt"
ikzf1 <- "ikzf1_narrowpeak.bed"
batf <- "batf_narrowpeak.bed"

nfkb<- "nfkb_narrowpeak.bed"
atf2 <- "atf2_narrowpeak.bed"

ctcf <- "GSE50893_SNYDER_HG38_GM12878_CTCF_VS_SNYDER_HG19_GM12878_INPUT_reconcile.dedup_peaks.encodePeak.txt"
sa1 <- "GSE50893_SNYDER_HG38_GM12878_SA1_VS_SNYDER_HG19_GM12878_INPUT_reconcile.dedup_peaks.encodePeak.txt"

epiread <- function(epifile){
    epiprofread <- fread(epifile)
    setnames(epiprofread, colnames(epiprofread[,1:3]), c("chr", "peak_start", "peak_end"))
    epiprofread[,chr := gsub("chr", "", chr)]
    return(epiprofread)
}

k27ac <- epiread(k27ac)
atac <- epiread(atac)

k36me3 <- epiread(k36me3)
ikzf1 <- epiread(ikzf1)
batf <- epiread(batf)

nfkb <- epiread(nfkb)
atf2 <- epiread(atf2)

ctcf <- epiread(ctcf)
sa1 <- epiread(sa1)

In [4]:
### loading baitmap file

## fres
bmap <- fread("hg38_dpnII.baitmap")


In [5]:
### gwas data
rmap_fres <- fread("hg38_dpnII.rmap")
rmap_5kb <- fread("human_DpnII_5K_sol_baits.rmap")

raw_deLange_data <- fread("28067908-GCST004132-EFO_0000384.h.tsv.gz")
ppi_deLange <- fread("CD_deLange_ILCs_hg38_ClassicCOGS_combinedInteractions_ALL/COGS_PPIs_data.table.txt")
ppi_deLange_susie <- fread("CD_deLange_ILCs_hg38_SuSIE_combinedInteractions_ALL/COGS_PPIs_data.table.txt")

delange_res <- fread("CD_deLange_ILCs_hg38_ClassicCOGS_combinedInteractions_ALL/Annotated_COGS_scores_data.table.txt")
delange_susie_res <- fread("CD_deLange_ILCs_hg38_SuSIE_combinedInteractions_ALL/Annotated_COGS_scores_data.table.txt")


In [7]:
## merging gwas data with rmap

ppirmap <- function(gwas, rmap_fres, rmap_5kb){
  
    gwas <- gwas[ppi >= 0.05]
    setnames(gwas, colnames(gwas[,1]), "V1")

    # overlap_by: "summit" or "region"
  
    # define middle point of a peak
    gwas[,mid := pos]
    gwas <- gwas[,c("V1", "mid", "pos")]
    gwas[,V1 := gsub("chr", "", V1)]
  
    # assign peaks to fres rmap bins 
    setkey(gwas, V1, mid, pos)
    setkey(rmap_fres, V1, V2, V3)
    gwas_rmap <- foverlaps(gwas, rmap_fres, by.x = c("V1", "mid", "pos"), by.y = c("V1", "V2", "V3"))
    gwas_rmap <- gwas_rmap[,c("V1", "mid", "pos", "V4")]
    setnames(gwas_rmap, "V4", "fres_ID")

    # assign peaks to 5kb rmap bins by summit
    setkey(gwas_rmap, V1, mid, pos)
    setkey(rmap_5kb, V1, V2, V3)
    gwas_rmap <- foverlaps(gwas_rmap, rmap_5kb, by.x = c("V1", "mid", "pos"), by.y = c("V1", "V2", "V3"))
    gwas_rmap <- gwas_rmap[,c("V1", "mid", "pos", "fres_ID", "V4")]
    setnames(gwas_rmap, "V4", "5kb_ID")
    
    gwas_rmap[,mid := NULL]
  
  return(gwas_rmap)
}

# assigning gwas snps to rmaps

ppi_deLange_rmap <- ppirmap(ppi_deLange, rmap_fres, rmap_5kb)
ppi_deLange_susie_rmap <- ppirmap(ppi_deLange_susie, rmap_fres, rmap_5kb)


In [8]:
## merging epigenetic data with rmap

epirmap <- function(epiprof, rmap_fres, rmap_5kb, overlap_by){
  
  if ("pos" %in% colnames(epiprof)){
    epiprof[,c("V2", "V3") := NULL]
    epiprof[,V2 := pos]
    epiprof[,V3 := pos]
  }else{
    setnames(epiprof, colnames(epiprof[,1:3]), c("V1", "V2", "V3"))
  }    
  
  # overlap_by: "summit" or "region"
  
  if (overlap_by == "summit"){
    
      # define middle point of a peak
      epiprof[,mid := (V2 + V3) /2]
      epiprof[,mid_2 := (V2 + V3) /2]
      epiprof <- epiprof[,c("V1", "V2", "V3", "mid", "mid_2")]
      epiprof[,V1 := gsub("chr", "", V1)]
  }else{
      # define middle point of a peak
      epiprof[,mid := V2]
      epiprof[,mid_2 := V3]
      epiprof <- epiprof[,c("V1", "V2", "V3", "mid", "mid_2")]
      epiprof[,V1 := gsub("chr", "", V1)]
  }

  
  # assign peaks to fres rmap bins by summit
  setkey(epiprof, V1, mid, mid_2)
  setkey(rmap_fres, V1, V2, V3)
  epi_rmap <- foverlaps(epiprof, rmap_fres, by.x = c("V1", "mid", "mid_2"), by.y = c("V1", "V2", "V3"))
  epi_rmap <- epi_rmap[,c("V1", "mid", "mid_2", "V4")]
  setnames(epi_rmap, "V4", "fres_ID")
  
  # assign peaks to 5kb rmap bins by summit
  setkey(epi_rmap, V1, mid, mid_2)
  setkey(rmap_5kb, V1, V2, V3)
  epi_rmap <- foverlaps(epi_rmap, rmap_5kb, by.x = c("V1", "mid", "mid_2"), by.y = c("V1", "V2", "V3"))
  epi_rmap <- epi_rmap[,c("V1", "mid", "mid_2", "fres_ID", "V4")]
  setnames(epi_rmap, "V4", "5kb_ID")

  
  return(epi_rmap)
}

# load rmaps
rmap_fres <- fread("/rds/general/user/malyshev/home/spivakov_analysis_live/Design/Human_DpnII_binsize1500_maxL75K/hg38_dpnII.rmap")
rmap_5kb <- fread("/rds/general/user/malyshev/home/spivakov_analysis_live/Design/Human_hg38_bin5K_sol_baits/human_DpnII_5K_sol_baits.rmap")

atac_rmap <- epirmap(atac, rmap_fres, rmap_5kb, "summit")
k27ac_rmap <- epirmap(k27ac, rmap_fres, rmap_5kb, "summit")


In [10]:
### prep mix resolution bedpe 

ilc_chic_abc_peakm_abc_only <- ilc_chic_abc_peakm[(is.na(chicago_fres) | chicago_fres <5) & (is.na(chicago_5kb) | chicago_5kb <5) &  ABC.Score >= 5]
ilc_chic_abc_peakm_chic5kb <- ilc_chic_abc_peakm[chicago_5kb >= 5]
ilc_chic_abc_peakm_chicfres <- ilc_chic_abc_peakm[(is.na(chicago_5kb) | chicago_5kb <5) & chicago_fres >= 5]

## don't need to collapse ilc_chic_abc_peakm_abc_only, as interactions are already unique
setnames(ilc_chic_abc_peakm_abc_only, "ABC.Score", "score")
ilc_chic_abc_peakm_abc_only <- ilc_chic_abc_peakm_abc_only[,c("baitChr", "baitStart", "baitEnd", "oeChr", "oeStart", "oeEnd", "score", "baitID", "oeID", "baitName")]
ilc_chic_abc_peakm_abc_only[, res_type := "ABC"]
ilc_chic_abc_peakm_abc_only[, delange_susie := ifelse(((baitID %in% ppi_deLange_susie_rmap$fres_ID) | (oeID %in% ppi_deLange_susie_rmap$fres_ID)), "TRUE", "FALSE")]
ilc_chic_abc_peakm_abc_only[, atac := ifelse(oeID %in% atac_rmap$fres_ID, "TRUE", "FALSE")]
ilc_chic_abc_peakm_abc_only[, k27ac := ifelse(oeID %in% k27ac_rmap$fres_ID, "TRUE", "FALSE")]


## collapse 5kb interactions
ilc_chic_abc_peakm_chic5kb_coll <- ilc_chic_abc_peakm_chic5kb[,list(baitChr = baitChr[1], baitStart = baitStart[1], baitEnd = baitStart[1],
                                                                    oeChr = oeChr[1], oeStart = min(oeStart), oeEnd = max(oeEnd), score = chicago_5kb[1], baitName = baitName[1]), 
                                                              by = c("baitID_5kb", "oeID_5kb")]
ilc_chic_abc_peakm_chic5kb_coll <- ilc_chic_abc_peakm_chic5kb_coll[,c("baitChr", "baitStart", "baitEnd", "oeChr", "oeStart", "oeEnd", "score", "baitID_5kb", "oeID_5kb", "baitName")]
setnames(ilc_chic_abc_peakm_chic5kb_coll, c("baitID_5kb", "oeID_5kb"), c("baitID", "oeID"))
ilc_chic_abc_peakm_chic5kb_coll[, res_type := "chicago_5kb"]
ilc_chic_abc_peakm_chic5kb_coll[, delange_susie := ifelse(((baitID %in% ppi_deLange_susie_rmap$'5kb_ID') | (oeID %in% ppi_deLange_susie_rmap$'5kb_ID')), "TRUE", "FALSE")]
ilc_chic_abc_peakm_chic5kb_coll[, atac := ifelse(oeID %in% atac_rmap$'5kb_ID', "TRUE", "FALSE")]
ilc_chic_abc_peakm_chic5kb_coll[, k27ac := ifelse(oeID %in% k27ac_rmap$'5kb_ID', "TRUE", "FALSE")]
               


## don't need to collapse ilc_chic_abc_peakm_abc_only, as interactions are already unique
setnames(ilc_chic_abc_peakm_chicfres, "chicago_fres", "score")
ilc_chic_abc_peakm_chicfres <- ilc_chic_abc_peakm_chicfres[,c("baitChr", "baitStart", "baitEnd", "oeChr", "oeStart", "oeEnd", "score", "baitID", "oeID", "baitName")]
ilc_chic_abc_peakm_chicfres[, res_type := "chicago_fres"]
ilc_chic_abc_peakm_chicfres[, delange_susie := ifelse(((baitID %in% ppi_deLange_susie_rmap$fres_ID) | (oeID %in% ppi_deLange_susie_rmap$fres_ID)), "TRUE", "FALSE")]
ilc_chic_abc_peakm_chicfres[, atac := ifelse(oeID %in% atac_rmap$fres_ID, "TRUE", "FALSE")]
ilc_chic_abc_peakm_chicfres[, k27ac := ifelse(oeID %in% k27ac_rmap$fres_ID, "TRUE", "FALSE")]
   

## merging
peakm_remerge <- rbind(ilc_chic_abc_peakm_chicfres, ilc_chic_abc_peakm_chic5kb_coll, ilc_chic_abc_peakm_abc_only)
setkey(peakm_remerge, baitChr, baitStart, baitEnd, oeChr, oeStart, oeEnd)

## colouring interactions (colourblind safe): ABC - #1b9e77,green; chicago_5kb - #d95f02,red ; chicago_fres - #7570b3, purple
peakm_remerge[,colour := ifelse(res_type == "ABC", "#1b9e77",
                               ifelse(res_type == "chicago_5kb", "#d95f02", "#7570b3"))]

peakm_remerge[, dist := ifelse(baitChr == oeChr, abs((oeEnd + oeStart)/2 - (baitStart + baitStart)/2), "NA")]

fwrite(peakm_remerge, "peakm_merged_chicago_fres_5kb_abc_fres_epiplot_lola.txt", sep = "\t", scipen = 999)

In [10]:
## plotting gene tracks

gene_track_plot <- function(gene_name = gene_name, chrom = chrom, bmap = bmap, bigStart = bigStart, bigEnd = bigEnd, midpoint = midpoint){
    
    mart <- useEnsembl(
    biomart = "ensembl", 
    dataset = "hsapiens_gene_ensembl"      
    )

    gene_annotation <- getBM(
      attributes = c("chromosome_name", "exon_chrom_start", "exon_chrom_end", "external_gene_name", "strand"),
      filters = c("chromosome_name", "start", "end"),
      values = list(chrom, bigStart*1000, bigEnd*1000),
      mart = mart
    )

    gene_annotation <- as.data.table(gene_annotation)

    gene_annotation[, `:=`(
      chromosome_name = as.character(chromosome_name),
      strand = as.character(strand)
    )]

    gene_annotation <- gene_annotation[, .(
      chrom = chromosome_name, 
      start = exon_chrom_start, 
      stop = exon_chrom_end, 
      gene = external_gene_name, 
      strand, 
      colour = adjustcolor("grey23", alpha.f = 0.5)
      #colour = ifelse(strand == 1, "darkgoldenrod", "dodgerblue4")
      )]

    for(current_gannot in gene_annotation[, sort(unique(gene))]){
          if(grepl(current_gannot, gene_name) == TRUE){
            gene_annotation[(gene == current_gannot), colour := "dodgerblue4"]
          }
        }

    gene_annotation[(strand == 1 | strand == -1), strand := ifelse(strand == 1, "+", "-")]
    gene_annotation <- gene_annotation[gene!=""]
        
    return(gene_annotation)

}
   

In [11]:
batch_epi_plot_short <- function(peakm, bait_list, bmap, window_left = window_left, window_right = window_right, zoom = zoom, all_ints = all_ints, idonly = TRUE){

    ## e.g. peakm can be ilc_chic_abc_peakm
    for (id in bait_list){
        
    gene_name <- bmap[V4 == id]$V5[1]

        if(zoom == TRUE){

          if (is.null(window_left)){
            window_left<- (min(as.numeric(peakm[baitID == id][baitChr == oeChr][delange_susie == TRUE]$dist)))/1000 - 20
          }else{
            window_left = window_left
          }

          if (is.null(window_right)){
            window_right<- (max(as.numeric(peakm[baitID == id][baitChr == oeChr][delange_susie == TRUE]$dist)))/1000 + 20
          }else{
            window_right = window_right
          }
        }else{
          if (is.null(window_left)){
            window_left<- (min(as.numeric(peakm[baitID == id][baitChr == oeChr]$dist)))/1000 - 20
          }else{
            window_left = window_left
          }

          if (is.null(window_right)){
            window_right<- (max(as.numeric(peakm[baitID == id][baitChr == oeChr]$dist)))/1000 + 20
          }else{
            window_right = window_right
          }
        }
        
        midpoint <- (bmap[V4==id]$V2+bmap[V4==id]$V3)/2000 # also convert to kb scale by dividing by 1000

        if(window_left<0){
          bigStart <- midpoint-abs(window_left)
        }else{
          bigStart <- midpoint-10
        }

        if(window_right>=0){
          bigEnd <- midpoint + abs(window_right)
        }else{
          bigEnd <- midpoint+10
        }
        
        chrom <- bmap[V4==id][1,]$V1
        ensg_id <- delange_susie_res[gene == gene_name]$ensg

        peakm_sel <- peakm[baitChr == chrom][((baitStart > bigStart*1000) & (oeEnd < bigEnd*1000)) | 
                                         ((oeStart > bigStart*1000) & (baitEnd < bigEnd*1000))]

        
        current_fn <- paste0("~/miniPCHiC/hILCs/hILCs_run2/git_ILCs_LOLA/LOLA_working_files/", 
                             gene_name, "_",id, "_", window_left, "kb_left_", window_right, "kb_right_zoom_", zoom, "all_ints_", all_ints, "_deLange_chic_abc_fres_epi_sushi.pdf")
        
        height <- 20
        npanels <- 13
        pdf(
          current_fn, 
          height = height, 
          width = height * 0.7070707070707# A4 proportions
        ) 
        layout(
          mat = matrix(c(1 : npanels), byrow = TRUE, nrow = npanels, ncol = 1), 
          heights = c(2, 1,1,1,1,1,1,1,1,1, 3, 3, 4)
        )
        par(mar = c(2, 4.1, 2, 4.1))
        

        #------------------------------------ Plot gene-track
        message("Plotting gene track")

        # Use plotGenes to plot the gene track
        
        gene_sel <- gene_track_plot(gene_name = gene_name, chrom = chrom, bmap = bmap, bigStart = bigStart, bigEnd = bigEnd, midpoint = midpoint)
        
        pgr <- plotGenes(
            gene_sel[, .(chrom, start, stop, gene, score = ".", strand)], 
            chrom = chrom,
            chromstart = bigStart*1000,
            chromend = bigEnd*1000,
            col = gene_sel$colour,
            bheight = 0.3, 
            lheight = 0.05,
            maxrows = 10000,
        #     types = "exon",
            plotgenetype = "arrow",
            wigglefactor = 0.05,
            labelat = "middle",
            #fontsize = 2
        )
        
        # Plot genome coordinate bar
        labelgenome(
          chrom = chrom,
          chromstart = bigStart*1000,
          chromend = bigEnd*1000,
          n = 10,
          scale = "Kb"
        )
        
         
        #------------------------------------
        # Plot ChIP-Seq pileups
        message("Plotting pileups")

        # message(paste0("Plotting pileup"))
        plotBed(
            beddata = atac,
            chrom = chrom,
            chromstart = bigStart*1000,
            chromend = bigEnd*1000,
            color = "grey16",
            ylab = "Read count",
            main = "ATAC",
            wiggle=0.0001,
            rowlabelcex = 2
        )
        
        # message(paste0("Plotting pileup"))
        plotBed(
            beddata = k27ac,
            chrom = chrom,
            chromstart = bigStart*1000,
            chromend = bigEnd*1000,
            color = "grey16",
            ylab = "Read count",
            rowlabels = "H3K27ac",
            rowlabelcol= "grey16",
            wiggle=0.0001,
            rowlabelcex = 2
        )
        
        # message(paste0("Plotting pileup"))
        if (nrow(k36me3[peak_start>bigStart & peak_end<bigEnd])>0) {
            
            plotBed(
                beddata = k36me3,
                chrom = chrom,
                chromstart = bigStart*1000,
                chromend = bigEnd*1000,
                color = "grey16",
                ylab = "Read count",
                rowlabels = "H3K36me3",
                rowlabelcol= "grey16",
                wiggle=0.0001,
                rowlabelcex = 2
        )

        }else { # If there are no significant interactions, plot an empty frame
              plotBedpe(
                bedpedata = data.frame(), 
                chrom = chrom, 
                chromstart = bigStart*1000, 
                chromend = bigEnd*1000, 
                bty = "o"
              )
        }
        
        # message(paste0("Plotting pileup"))
        plotBed(beddata = ctcf,
            chrom = chrom,
            chromstart = bigStart*1000,
            chromend = bigEnd*1000,
            color = "grey16",
            ylab = "Read count",
            rowlabels = "CTCF",
            rowlabelcol= "grey16",
            wiggle=0.0001,
            rowlabelcex = 2
        )
        
        # message(paste0("Plotting pileup"))
        plotBed(beddata = sa1,
            chrom = chrom,
            chromstart = bigStart*1000,
            chromend = bigEnd*1000,
            color = "grey16",
            ylab = "Read count",
            rowlabels = "SA1",
            rowlabelcol= "grey16",
            wiggle=0.0001,
            rowlabelcex = 2
        )
        
        # message(paste0("Plotting pileup"))
        plotBed(beddata = ikzf1,
          chrom = chrom,
          chromstart = bigStart*1000,
          chromend = bigEnd*1000,
          color = "grey16",
          ylab = "Read count",
            rowlabels = "IKZF1",
            rowlabelcol= "grey16",
            wiggle=0.0001,
            rowlabelcex = 2
        )
        
        # message(paste0("Plotting pileup"))
        plotBed(beddata = batf,
          chrom = chrom,
          chromstart = bigStart*1000,
          chromend = bigEnd*1000,
          color = "grey16",
          ylab = "Read count",
            rowlabels = "BATF",
            rowlabelcol= "grey16",
            wiggle=0.0001,
            rowlabelcex = 2
        )
        
        # message(paste0("Plotting pileup"))
        plotBed(beddata = nfkb,
          chrom = chrom,
          chromstart = bigStart*1000,
          chromend = bigEnd*1000,
          color = "grey16",
          ylab = "Read count",
            rowlabels = "NFKB",
            rowlabelcol= "grey16",
            wiggle=0,
            rowlabelcex = 2
        )
        
        # message(paste0("Plotting pileup"))
        plotBed(beddata = atf2,
          chrom = chrom,
          chromstart = bigStart*1000,
          chromend = bigEnd*1000,
          color = "grey16",
          ylab = "Read count",
            rowlabels = "ATF2",
            rowlabelcol= "grey16",
            wiggle=0.0001,
            rowlabelcex = 2
        )

        #------------------------------------
        # Plot SNP posterior probabilities that intersect the plotting window

        message("Plotting de Lange SNP posteriors")
        ppi_deLange_sel <- ppi_deLange[chr ==chrom][(pos > bigStart*1000) & (pos < bigEnd*1000)]
        plot(
            x = ppi_deLange_sel[, pos], 
            y = ppi_deLange_sel[, ppi],
            cex = 2,
            xaxs = "i",
            xlim = c(bigStart*1000, bigEnd*1000),
            ylim = c(0, max(ppi_deLange_sel$ppi)),
            xlab = NA,
            ylab = "Posterior probability of variant being causal",
            col = adjustcolor("black", alpha.f = 0.9)
        )

        #------------------------------------
        # Plot SNP posterior probabilities that intersect the plotting window
        message("Plotting de Lange SuSiE SNP posteriors")
        ppi_deLange_susie_sel <- ppi_deLange_susie[chr ==chrom][(pos > bigStart*1000) & (pos < bigEnd*1000)]
        plot(
            x = ppi_deLange_susie_sel[, pos],
            y = ppi_deLange_susie_sel[, ppi],
            xlim = c(bigStart*1000, bigEnd*1000),
            ylim = c(0, max(ppi_deLange_susie_sel$ppi)),
            cex = 2, 
            xaxs = "i",
            xlab = NA,
            ylab = "Posterior probability of variant being causal",
            col = adjustcolor("black", alpha.f = 0.9)
        )

       
        #------------------------------------
        # Plot Promoter interactions that intersect the plotting window
        
        message("Plotting PCHiC arcs")
        
        
        if(nrow(peakm_sel[baitName == gene_name]) > 0){

            if (all_ints == TRUE){
                plotBedpe(
                    bedpedata = peakm_sel, 
                    chrom = chrom, 
                    chromstart = bigStart*1000, 
                    chromend = bigEnd*1000,
                    color = peakm_sel$colour,
                    heights = peakm_sel$score, 
                    flip = TRUE, 
                    plottype = "loops", 
                    bty = "o",
                    main = "promoter interactions in ILC3a"
              )
            }else{

               plotBedpe(
                    bedpedata = peakm_sel[baitName == gene_name], 
                    chrom = chrom, 
                    chromstart = bigStart*1000, 
                    chromend = bigEnd*1000,
                    color = peakm_sel[baitName == gene_name]$colour,
                    heights = peakm_sel[baitName == gene_name]$score, 
                    flip = TRUE, 
                    plottype = "loops", 
                    bty = "o",
                    main = "promoter interactions in ILC3s"
                   )
            }
            
        } else { # If there are no significant interactions, plot an empty frame
              plotBedpe(
                bedpedata = data.frame(), 
                chrom = chrom, 
                chromstart = bigStart, 
                chromend = bigEnd, 
                bty = "o"
              )
        }        
        
        # Plot genome coordinate bar
        labelgenome(
          chrom = chrom,
          chromstart = bigStart*1000,
          chromend = bigEnd*1000,
          n = 10,
          scale = "Kb"
        )
        
        #------------------------------------ Highlighting baits
        message("Highlighting baits")
        
        par(mfrow = c(1, 1))
        bmap_sel <- bmap[V1 == chrom][V2>bigStart*1000 & V3<bigEnd*1000]
        rect(
          xleft = bmap_sel[, V2], 
          xright = bmap_sel[, V3],
          ybottom = -50, 
          ytop = 20, 
          col = adjustcolor("steelblue", alpha.f = 0.3),
          border = NA
          )
        
        #----------------------------------
        # Add vertical bar over the PIRs with high PPis region
        message("Highlighting PIRs with PPis")
        
        par(mfrow = c(1, 1))
        rect(
          xleft = peakm_sel[baitName == gene_name][delange_susie == TRUE][, oeStart], 
          xright = peakm_sel[baitName == gene_name][delange_susie == TRUE][, oeEnd],
          ybottom = -50, 
          ytop = 20, 
          col = adjustcolor("tomato3", alpha.f = 0.3),
          border = NA
          )

        
        dev.off()
        
    }
        window_left <- NULL
        window_right <- NULL
    
}

In [55]:
suppressWarnings(batch_epi_plot_short(peakm_remerge, 2314200, bmap, window_left = -30, window_right = 30, zoom = TRUE, all_ints = TRUE))


Plotting gene track

Cache found



[1] "yes"


Plotting pileups



[1] "yes"
[1] "yes"


Plotting de Lange SNP posteriors

Plotting de Lange SuSiE SNP posteriors

Plotting PCHiC arcs



[1] "yes"


Highlighting baits

Highlighting PIRs with PPis



In [12]:
suppressWarnings(batch_epi_plot_short(peakm_remerge, 5834362, bmap, window_left = -100, window_right = 20, zoom = TRUE, all_ints = TRUE))


Plotting gene track

Cache found



[1] "yes"


Plotting pileups



[1] "yes"
[1] "yes"


Plotting de Lange SNP posteriors

Plotting de Lange SuSiE SNP posteriors

Plotting PCHiC arcs



[1] "yes"


Highlighting baits

Highlighting PIRs with PPis



In [11]:
suppressWarnings(batch_epi_plot_short(peakm_remerge, 3270636, bmap, window_left = -150, window_right = 10, zoom = TRUE, all_ints = FALSE))


Plotting gene track

Cache found



[1] "yes"


Plotting pileups



[1] "yes"
[1] "yes"


Plotting de Lange SNP posteriors

Plotting de Lange SuSiE SNP posteriors

Plotting PCHiC arcs



[1] "yes"


Highlighting baits

Highlighting PIRs with PPis



In [18]:
eqtl <- fread("~/miniPCHiC/hILCs/hILCs_run2/eQTL_data/Lepik_2017_ge_blood.all.tsv.gz")


In [39]:
head(eqtl)

molecular_trait_id,chromosome,position,ref,alt,variant,ma_samples,maf,pvalue,beta,se,type,ac,an,r2,molecular_trait_object_id,gene_id,median_tpm,rsid
<chr>,<int>,<int>,<chr>,<chr>,<chr>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<int>,<int>,<lgl>,<chr>,<chr>,<dbl>,<chr>
ENSG00000187583,1,10492,C,T,chr1_10492_C_T,73,0.0774947,0.7272200,0.0418361,0.1198600,SNP,73,942,NA,ENSG00000187583,ENSG00000187583,0.925,rs55998931
ENSG00000187608,1,10492,C,T,chr1_10492_C_T,73,0.0774947,0.0143885,-0.2756400,0.1121940,SNP,73,942,NA,ENSG00000187608,ENSG00000187608,191.429,rs55998931
ENSG00000187961,1,10492,C,T,chr1_10492_C_T,73,0.0774947,0.2525570,0.0914901,0.0798615,SNP,73,942,NA,ENSG00000187961,ENSG00000187961,10.698,rs55998931
ENSG00000188290,1,10492,C,T,chr1_10492_C_T,73,0.0774947,0.4037420,-0.1028980,0.1231230,SNP,73,942,NA,ENSG00000188290,ENSG00000188290,8.078,rs55998931
ENSG00000188976,1,10492,C,T,chr1_10492_C_T,73,0.0774947,0.8356740,-0.0129690,0.0624863,SNP,73,942,NA,ENSG00000188976,ENSG00000188976,30.355,rs55998931
ENSG00000225880,1,10492,C,T,chr1_10492_C_T,73,0.0774947,0.2337480,0.1343220,0.1126540,SNP,73,942,NA,ENSG00000225880,ENSG00000225880,2.120,rs55998931


In [45]:
batch_epi_plot_short <- function(peakm, bait_list, bmap, window_left = window_left, window_right = window_right, zoom = zoom, all_ints = all_ints, idonly = TRUE){

    ## e.g. peakm can be ilc_chic_abc_peakm
    for (id in bait_list){
        
    gene_name <- bmap[V4 == id]$V5[1]

        if(zoom == TRUE){

          if (is.null(window_left)){
            window_left<- (min(as.numeric(peakm[baitID == id][baitChr == oeChr][delange_susie == TRUE]$dist)))/1000 - 20
          }else{
            window_left = window_left
          }

          if (is.null(window_right)){
            window_right<- (max(as.numeric(peakm[baitID == id][baitChr == oeChr][delange_susie == TRUE]$dist)))/1000 + 20
          }else{
            window_right = window_right
          }
        }else{
          if (is.null(window_left)){
            window_left<- (min(as.numeric(peakm[baitID == id][baitChr == oeChr]$dist)))/1000 - 20
          }else{
            window_left = window_left
          }

          if (is.null(window_right)){
            window_right<- (max(as.numeric(peakm[baitID == id][baitChr == oeChr]$dist)))/1000 + 20
          }else{
            window_right = window_right
          }
        }
        
        midpoint <- (bmap[V4==id]$V2+bmap[V4==id]$V3)/2000 # also convert to kb scale by dividing by 1000

        if(window_left<0){
          bigStart <- midpoint-abs(window_left)
        }else{
          bigStart <- midpoint-10
        }

        if(window_right>=0){
          bigEnd <- midpoint + abs(window_right)
        }else{
          bigEnd <- midpoint+10
        }
        
        chrom <- bmap[V4==id][1,]$V1
        ensg_id <- delange_susie_res[gene == gene_name]$ensg

        peakm_sel <- peakm[baitChr == chrom][((baitStart > bigStart*1000) & (oeEnd < bigEnd*1000)) | 
                                         ((oeStart > bigStart*1000) & (baitEnd < bigEnd*1000))]

        eqtl_sel_CLN3 <- eqtl[gene_id == ensg_id][chromosome == chrom][position/1000> bigStart & position/1000<bigEnd]
 
        
        current_fn <- paste0("~/miniPCHiC/hILCs/hILCs_run2/git_ILCs_LOLA/LOLA_working_files/", 
                             gene_name, "_",id, "_", window_left, "kb_left_", window_right, "kb_right_zoom_", zoom, "all_ints_", all_ints, "_deLange_chic_abc_fres_epi_sushi.pdf")
        
        height <- 20
        npanels <- 14
        pdf(
          current_fn, 
          height = height, 
          width = height * 0.7070707070707# A4 proportions
        ) 
        layout(
          mat = matrix(c(1 : npanels), byrow = TRUE, nrow = npanels, ncol = 1), 
          heights = c(2, 1,1,1,1,1,1,1,1,1, 3, 3, 3, 3, 4)
        )
        par(mar = c(2, 4.1, 2, 4.1))
        

        #------------------------------------ Plot gene-track
        message("Plotting gene track")

        # Use plotGenes to plot the gene track
        
        gene_sel <- gene_track_plot(gene_name = gene_name, chrom = chrom, bmap = bmap, bigStart = bigStart, bigEnd = bigEnd, midpoint = midpoint)
        
        pgr <- plotGenes(
            gene_sel[, .(chrom, start, stop, gene, score = ".", strand)], 
            chrom = chrom,
            chromstart = bigStart*1000,
            chromend = bigEnd*1000,
            col = gene_sel$colour,
            bheight = 0.3, 
            lheight = 0.05,
            maxrows = 10000,
        #     types = "exon",
            plotgenetype = "arrow",
            wigglefactor = 0.05,
            labelat = "middle",
            #fontsize = 2
        )
        
        # Plot genome coordinate bar
        labelgenome(
          chrom = chrom,
          chromstart = bigStart*1000,
          chromend = bigEnd*1000,
          n = 10,
          scale = "Kb"
        )
        
         
        #------------------------------------
        # Plot ChIP-Seq pileups
        message("Plotting pileups")

        # message(paste0("Plotting pileup"))
        plotBed(
            beddata = atac,
            chrom = chrom,
            chromstart = bigStart*1000,
            chromend = bigEnd*1000,
            color = "grey16",
            ylab = "Read count",
            rowlabels = "ATAC",
            wiggle=0.0001,
            rowlabelcex = 2
        )
        
        # message(paste0("Plotting pileup"))
        plotBed(
            beddata = k27ac,
            chrom = chrom,
            chromstart = bigStart*1000,
            chromend = bigEnd*1000,
            color = "grey16",
            ylab = "Read count",
            rowlabels = "H3K27ac",
            rowlabelcol= "grey16",
            wiggle=0.0001,
            rowlabelcex = 2
        )
        
#         # message(paste0("Plotting pileup"))
#         if (nrow(k36me3[chr == chrom][peak_start>bigStart & peak_end<bigEnd])>0) {
            
#             plotBed(
#                 beddata = k36me3,
#                 chrom = chrom,
#                 chromstart = bigStart*1000,
#                 chromend = bigEnd*1000,
#                 color = "grey16",
#                 ylab = "Read count",
#                 rowlabels = "H3K36me3",
#                 rowlabelcol= "grey16",
#                 wiggle=0.0001,
#                 rowlabelcex = 2
#                 )

#         }else { # If there are no significant interactions, plot an empty frame
#               plotBedpe(
#                 bedpedata = data.frame(), 
#                 chrom = chrom, 
#                 chromstart = bigStart*1000, 
#                 chromend = bigEnd*1000, 
#                 bty = "o"
#               )
#         }
        
        plotBed(beddata = k36me3,
                chrom = chrom,
                chromstart = bigStart*1000,
                chromend = bigEnd*1000,
                color = "grey16",
                ylab = "Read count",
                rowlabels = "H3K36me3",
                rowlabelcol= "grey16",
                wiggle=0.0001,
                rowlabelcex = 2
                )
        
        # message(paste0("Plotting pileup"))
        plotBed(beddata = ctcf,
            chrom = chrom,
            chromstart = bigStart*1000,
            chromend = bigEnd*1000,
            color = "grey16",
            ylab = "Read count",
            rowlabels = "CTCF",
            rowlabelcol= "grey16",
            wiggle=0.0001,
            rowlabelcex = 2
        )
        
        # message(paste0("Plotting pileup"))
        plotBed(beddata = sa1,
            chrom = chrom,
            chromstart = bigStart*1000,
            chromend = bigEnd*1000,
            color = "grey16",
            ylab = "Read count",
            rowlabels = "SA1",
            rowlabelcol= "grey16",
            wiggle=0.0001,
            rowlabelcex = 2
        )
        
        # message(paste0("Plotting pileup"))
        plotBed(beddata = ikzf1,
          chrom = chrom,
          chromstart = bigStart*1000,
          chromend = bigEnd*1000,
          color = "grey16",
          ylab = "Read count",
            rowlabels = "IKZF1",
            rowlabelcol= "grey16",
            wiggle=0.0001,
            rowlabelcex = 2
        )
        
        # message(paste0("Plotting pileup"))
        plotBed(beddata = batf,
          chrom = chrom,
          chromstart = bigStart*1000,
          chromend = bigEnd*1000,
          color = "grey16",
          ylab = "Read count",
            rowlabels = "BATF",
            rowlabelcol= "grey16",
            wiggle=0.0001,
            rowlabelcex = 2
        )
        
        # message(paste0("Plotting pileup"))
        plotBed(beddata = nfkb,
          chrom = chrom,
          chromstart = bigStart*1000,
          chromend = bigEnd*1000,
          color = "grey16",
          ylab = "Read count",
            rowlabels = "NFKB",
            rowlabelcol= "grey16",
            wiggle=0,
            rowlabelcex = 2
        )
        
        # message(paste0("Plotting pileup"))
        plotBed(beddata = atf2,
          chrom = chrom,
          chromstart = bigStart*1000,
          chromend = bigEnd*1000,
          color = "grey16",
          ylab = "Read count",
            rowlabels = "ATF2",
            rowlabelcol= "grey16",
            wiggle=0.0001,
            rowlabelcex = 2
        )

        message("Plotting eQTL data")
        
        plot(
            x = eqtl_sel_CLN3[chromosome == chrom][, position], 
            y = eqtl_sel_CLN3[chromosome == chrom][, (-1)*log(pvalue, 10)],
            cex = 2,
            xaxs = "i",
            xlim = c(bigStart*1000, bigEnd*1000),
            ylim = c(0, 13),
            xlab = NA,
            ylab = "eQTL for CLN3 in blood (Lepik et al., 2017)",
            col = adjustcolor("black", alpha.f = 0.9)
        )
        
        #------------------------------------
        # Plot SNP posterior probabilities that intersect the plotting window
        message("Plotting raw CD data posteriors")
        
        plot(
            x = raw_deLange_data[hm_chrom == chrom][, hm_pos], 
            y = raw_deLange_data[hm_chrom == chrom][, (-1)*log(p_value, 10)],
            cex = 2,
            xaxs = "i",
            xlim = c(bigStart*1000, bigEnd*1000),
            ylim = c(0, max((-1)*log(raw_deLange_data[hm_chrom == chrom][(hm_pos > bigStart*1000) & (hm_pos < bigEnd*1000)]$p_value, 10))),
            xlab = NA,
            ylab = "CD GWAS -log10(p-value)",
            col = adjustcolor("black", alpha.f = 0.9)
        )

        #------------------------------------
        # Plot SNP posterior probabilities that intersect the plotting window
        message("Plotting de Lange SuSiE SNP posteriors")
        ppi_deLange_susie_sel <- ppi_deLange_susie[chr ==chrom][(pos > bigStart*1000) & (pos < bigEnd*1000)]
        plot(
            x = ppi_deLange_susie_sel[chr ==chrom][, pos],
            y = ppi_deLange_susie_sel[chr ==chrom][, ppi],
            xlim = c(bigStart*1000, bigEnd*1000),
            ylim = c(0, max(ppi_deLange_susie_sel[chr ==chrom]$ppi)),
            cex = 2, 
            xaxs = "i",
            xlab = NA,
            ylab = "Posterior probability of variant being causal",
            col = adjustcolor("black", alpha.f = 0.9)
        )

        

        #------------------------------------
        # Plot Promoter interactions that intersect the plotting window
        
        message("Plotting PCHiC arcs")
        
        
        if(nrow(peakm_sel[baitName == gene_name]) > 0){

            if (all_ints == TRUE){
                plotBedpe(
                    bedpedata = peakm_sel, 
                    chrom = chrom, 
                    chromstart = bigStart*1000, 
                    chromend = bigEnd*1000,
                    color = peakm_sel$colour,
                    heights = peakm_sel$score, 
                    flip = TRUE, 
                    plottype = "loops", 
                    bty = "o",
                    main = "promoter interactions in ILC3a"
              )
            }else{

               plotBedpe(
                    bedpedata = peakm_sel[baitName == gene_name], 
                    chrom = chrom, 
                    chromstart = bigStart*1000, 
                    chromend = bigEnd*1000,
                    color = peakm_sel[baitName == gene_name]$colour,
                    heights = peakm_sel[baitName == gene_name]$score, 
                    flip = TRUE, 
                    plottype = "loops", 
                    bty = "o",
                    main = "promoter interactions in ILC3s"
                   )
            }
            
        } else { # If there are no significant interactions, plot an empty frame
              plotBedpe(
                bedpedata = data.frame(), 
                chrom = chrom, 
                chromstart = bigStart, 
                chromend = bigEnd, 
                bty = "o"
              )
        }        
        
        # Plot genome coordinate bar
        labelgenome(
          chrom = chrom,
          chromstart = bigStart*1000,
          chromend = bigEnd*1000,
          n = 10,
          scale = "Kb"
        )
        
        #------------------------------------ Highlighting baits
        message("Highlighting baits")
        
        par(mfrow = c(1, 1))
        bmap_sel <- bmap[V1 == chrom][V2>bigStart*1000 & V3<bigEnd*1000]
        rect(
          xleft = bmap_sel[, V2], 
          xright = bmap_sel[, V3],
          ybottom = -50, 
          ytop = 20, 
          col = adjustcolor("steelblue", alpha.f = 0.3),
          border = NA
          )
        
        #----------------------------------
        # Add vertical bar over the PIRs with high PPis region
        message("Highlighting PIRs with PPis")
        
        par(mfrow = c(1, 1))
        rect(
          xleft = peakm_sel[baitName == gene_name][delange_susie == TRUE][, oeStart], 
          xright = peakm_sel[baitName == gene_name][delange_susie == TRUE][, oeEnd],
          ybottom = -50, 
          ytop = 20, 
          col = adjustcolor("tomato3", alpha.f = 0.3),
          border = NA
          )

        
        dev.off()
        
    }
        window_left <- NULL
        window_right <- NULL
    
}

In [46]:
suppressWarnings(batch_epi_plot_short(peakm_remerge, 2314200, bmap, window_left = -30, window_right = 30, zoom = TRUE, all_ints = TRUE))


Plotting gene track

Cache found



[1] "yes"


Plotting pileups



[1] "yes"
[1] "yes"


Plotting eQTL data

Plotting raw CD data posteriors

Plotting de Lange SuSiE SNP posteriors

Plotting PCHiC arcs



[1] "yes"


Highlighting baits

Highlighting PIRs with PPis

